## Problems

1. **The current issue with the 7 day forecast is the following**: ✔;


2. **Dates are not updated into the graph. Can I integrate dates into the training?** --> Done ✔


3. **The final 7 predictions to the 7 day forecast seems to be missing.** Could that be an issue with how the data is split and divided? --> Done, applying the dates with the graph helped resolve this issue.✔


4. **One consideration I didn't think of.** I have it configured so that a time series of 20 is used to predict 1 to 7 days ahead. I only properly recorded the days for seven days ahead. How do I account for this in the other days? --> Done, will now compare a 7 day prediction vs a 7th day prediction✔


5. **I need to see how dates are handled** considering that each prediction has seven dates, ranging from 1 to seven days into the future. -->Done✔


6. **I'm sure the dates are incorrect** due to this seven day b.s.--> Done✔


7. **How is mean squared error calculated.** Will ask ChatGPT about this. --> done✔


8. **Can I calculate RMSE for all seven days?** --> done✔


9. **I'm thinking that the implementation in which only the seventh day variable is predicted** might be more accurate than its 7 day individual implementation.--> done, early testing seems to indicate that both solutions are not predicting well enough for profit.✔


10. **I just Realized!**, is the training happening to all seven dates? That could explain the odd skewing that's occurring if the training is happening on one date. --> done, the skewing was the result of different size graphs appearing to have skewed.✔


11. **Need to compare the single date trained model with the multiple date trained model.** This should be interesting.--> Done, not much difference.✔


12. **Now we need to see how I can improve the lag between 7 day predictions and the original values.** Options to include is either hyperparameter optimization or/and using different types of recurrent neural networks. Perhaps look into alternative machine learning paradigms such as LSTM and Transformers and models with attention mechanisms. Non-linear Autoregressive with Exogenous Inputs (NARX) model

13. **Eventually, we will want to use trained models to predict closing prices on future dates.** We need to figure out how to do this on days in which the stock market will deifnitely be open. Maybe there is a collection of future dates that provides this? 

14. **Is RMSE values the only metric we have to go by?** Perhaps there a more metrics we can consider. One metric that would be phenomenal is to consider how much money would have been won/lost if we were to attempt to predict stock prices based on past data. 


## Solutions

1. **Before I solve problem 3**, let's see if I can put the dates appropriately into the graph. Then I will go about solving problem 2 to solve the last values of the graph. --> Done ✔

2. To solve the issue between problems 4 to 6:
    - a. **I need to record the dates for 7 outcomes ranging from 1 to 7.** I'm thinking maybe I need to apply data structures of data structures. --> List of lists. One data structure will house the last day of the look back sequence, and then connected to that date, would be the following seven predicted dates. --> Instead of indexed by dates, it's just indexed by the proper date amounts. --> Done ✔

    - b. **First, let's ask ChatGPT** to see what solution it should implement by pasting the function, and bringing up the fact that only the final of the seven dates are being recorded. If it doesn't make something good, then I will apply 2c. --> Done✔

    - c. **Have ChatGPT apply the solution from 2a.** --> Done✔

3. To resolve problems 9 and 10:
    - a. **I will now attempt to resolve problems by training on dates individually.** First training day 7. And then seeing if the accuracy is much better. --> Done✔

    - b. **A solution I'm considering is altering the function** so that I can put in a forecast number and it will output exclusively data and dates accordingly. --> Done✔

4. To resolve problem 12:
    - a. **First Let's see the effect of changing lookback has on the data.** Maybe this can improve rmse values, or ther metrics i decide along the way.

    -b. **Let's figure out if there is a way I can generate images to help visualize any change in data.**

5. Before solution 4, let's consider resolving problem 14. 
    - a. **If I can figure out how to best showcase profit** I can see if hyperparemeter modification is truly going something. I just considered something. I can just consider the profitability of the stock based on an algorithm i can apply. For the sake of argument i'm going to say we are working with seven days, but the algorithm will ultimately be made to adapt from 1 to 7 days. Anyways  The algorithm looks seven days ahead on the predicted graph. If it predicts the stock will be higher than the current actual price, it will buy one stock worth in anticipation of selling at the seven day mark. I will then sell it at the seven day mark, and the loss/gain will be noted. 

    - b. **Another more complicated algorithm to consider is** the same as above, but once it reaches the seven day mark, it will keep buying if it predicts higher, but then sell once it predicts the stock will get lower. 

    - c. **Yet another variation** is we just buy once, but then we sell once the stock is predicted to get much lower.

    - d. **one thing to consider is to apply a theshold of when to buy or sell stocks.** As in once stocks are predicted to rise or fall past a percentage in comparison to current price, that will prompt purchasing or selling. 

    - e. **Fianlly we can see if papers apply any solutions to this**

    - f. **I want to consider two algorithms**. Both will apply 


# Work in Process Algorithm for solution 5

```python
def trade_with_strategy(predicted_prices, actual_prices, strategy='A', buy_sell_threshold=0.05):
    """
    Implements a trading strategy based on predicted and actual stock prices.

    Parameters:
    - predicted_prices: List of predicted stock prices.
    - actual_prices: List of actual stock prices.
    - strategy: Trading strategy to use ('A', 'B', or 'C').
    - buy_sell_threshold: Threshold for buying/selling based on price change percentage (for strategy 'C').

    Returns:
    - Final cash amount after executing the trading strategy.
    - List of transaction tuples (action, shares, price).
    """
    cash = 10000  # Starting cash
    shares = 0
    transactions = []

    for i in range(len(predicted_prices) - 7):
        current_price = actual_prices[i]
        predicted_future_price = predicted_prices[i + 7]

        # Buy condition
        if predicted_future_price > current_price:
            shares_bought = cash // current_price
            cash -= shares_bought * current_price
            shares += shares_bought
            transactions.append(('BUY', shares_bought, current_price))

        # Sell conditions based on strategy
        if strategy == 'A' or (strategy == 'B' and (i + 7 < len(predicted_prices) and predicted_prices[i + 7] < predicted_prices[i])):
            cash += shares * actual_prices[i + 7]
            if shares > 0:
                transactions.append(('SELL', shares, actual_prices[i + 7]))
            shares = 0
        elif strategy == 'C':
            if shares > 0 and ((predicted_future_price - current_price) / current_price) >= buy_sell_threshold:
                cash += shares * actual_prices[i + 7]
                transactions.append(('SELL', shares, actual_prices[i + 7]))
                shares = 0

    # Final sell off at the last available price
    if shares > 0:
        cash += shares * actual_prices[-1]
        transactions.append(('SELL', shares, actual_prices[-1]))
        shares = 0

    return cash, transactions
```

This code defines a function `trade_with_strategy` that simulates a trading strategy over a given period based on predicted and actual stock prices. It supports three strategies:
- **Strategy A**: Sell after 7 days regardless.
- **Strategy B**: Sell earlier if a future prediction suggests a price drop.
- **Strategy C**: Sell based on a buy/sell threshold determined by the percentage change in price.

The function starts with an initial cash amount, iterates over the price lists to make buy/sell decisions, and tracks these transactions. The final cash amount and a list of transactions are returned.



Place holder for futre images

![Stock Prediction Graph](Images\decomposition_stocks_CHPT.png "Stock Prediction")